# Top Charts requests

## Setup

In [ ]:
# Import modules
import os

import pandas as pd

from datetime import datetime, timedelta
from tqdm import tqdm
from soundcharts_API_client import *


In [ ]:
# API connexion
x_app_id = ''
x_api_key = ''

sdch = SoundchartsClient(x_app_id, x_api_key)


## Get playlists

In [ ]:
# Get top charts playlists
results_df = []

offset = 0
limit = 100
more_playlists = True

while more_playlists:
    response = sdch.get_playlists_by_type(offset=offset, limit=limit, type='charts')

    # Extract playlist details
    playlists = response.get("items", [])
    for item in playlists:
        playlist_id = item.get("uuid")
        playlist_name = item.get("name")
        playlist_type = item.get("type")

        # Append the playlist data to the list
        results_df.append({
            "playlist_id": playlist_id,
            "playlist_name": playlist_name,
            "playlist_type": playlist_type,
        })

    # Update the offset to fetch the next set of playlists
    offset += limit

    # If the number of returned items is less than the limit, stop the loop
    if len(playlists) < limit:
        more_playlists = False

# Convert the list to a DataFrame
df_1 = pd.DataFrame(results_df)
df_1.to_excel('charts_playlists.xlsx', index=False)

# Print the DataFrame
display(df_1)


## Get playlists' tracklist

In [ ]:
# Import playlist data
df = pd.read_excel('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/db_soundcharts/charts_playlists.xlsx')


In [ ]:
start_date = datetime.strptime('2021-10-01T00:00:00+00:00', '%Y-%m-%dT%H:%M:%S+00:00')
end_date = datetime.strptime('2022-09-30T00:00:00+00:00', '%Y-%m-%dT%H:%M:%S+00:00')
days_range = (end_date - start_date).days
print(days_range)

# Get the data
# Define the output file
output_file = "charts_playlists_tracks_21-22.csv"

# Check if the file already exists and load existing data
if os.path.exists(output_file):
    api_results_df = pd.read_csv(output_file)
else:
    api_results_df = pd.DataFrame()

# Create a set of already processed track IDs to avoid duplicates
processed_playists = set(api_results_df['playlist_id'].unique()) if not api_results_df.empty else set()

# List to store new results temporarily
new_results = []

for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
    playlist_id = row['playlist_id']
    playlist_name = row['playlist_name']
    playlist_type = row['playlist_type']

    for i in range(days_range + 1):
        try:
            fetch_date = (end_date - timedelta(days = days_range - i)).strftime('%Y-%m-%dT%H:%M:%S+00:00')

            offset = 0
            limit = 100
            more_tracks = True

            while more_tracks:
                try:
                    response = sdch.get_playlist_tracklist_by_date(
                        offset=offset, 
                        limit=limit, 
                        playlist_id=playlist_id, 
                        date=fetch_date
                    )

                    # Extract collection date
                    collection_date = response.get("related", {}).get("date")

                    # Extract track items and loop through them
                    tracks = response.get("items", [])
                    for item in tracks:
                        track_position = item.get("position")
                        track_name = item.get("song", {}).get("name")
                        track_uuid = item.get("song", {}).get("uuid")
                    
                        # Append the track data to the list
                        playlist_data = ({
                            "playlist_id": playlist_id,
                            "playlist_name": playlist_name,
                            "collection_date": collection_date,
                            "playlist_type": playlist_type,
                            "track_position": track_position,
                            "track_name": track_name,
                            "track_uuid": track_uuid,
                        })

                        new_results.append(playlist_data)

                        pd.DataFrame([playlist_data]).to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)

                    # Update the offset to fetch the next set of tracks
                    offset += limit

                    # If the number of returned items is less than the limit, stop the loop
                    if len(tracks) < limit:
                        more_tracks = False

                except Exception as e:
                    print(f"Error processing offset {offset} for playlist {playlist_name}: {e}")
                    continue
        
        except Exception as e:
            print(f"Error fetching date {fetch_date} for playlist {playlist_name}: {e}")
            continue

# Convert the new results to a DataFrame and combine with existing data if needed
if new_results:
    new_results_df = pd.DataFrame(new_results)
    api_results_df = pd.concat([api_results_df, new_results_df], ignore_index=True)

# Print the DataFrame
display(api_results_df)


## Get tracks and followers info 

In [21]:
# Import playlist data
df = pd.read_csv('/Users/julienmbarki/Documents/Doctorat/Publications/Article 2/Data/Code/data_management/db_soundcharts/charts_playlists_tracks_23-24.csv')


In [22]:
# Find the first appearance of each unique track
df['collection_date_clean'] = pd.to_datetime(df['collection_date'])
first_appearance_df = df.sort_values(by='collection_date_clean').groupby('track_uuid').first().reset_index()

# print the first appearance of each unique track
display(first_appearance_df)


,track_uuid,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,collection_date_clean
0,000403f8-13a8-42af-8df6-f97b0f5372c7,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-31T00:00:00+00:00,Charts,21,CARLO,2023-10-31 00:00:00+00:00
1,000e110b-3397-4082-982c-d85b0636629f,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-05-02T00:00:00+00:00,Charts,40,L o s e M y M i n d,2024-05-02 00:00:00+00:00
2,002bed21-2779-4169-a6bc-9b57cbb437c4,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-11-13T00:00:00+00:00,Charts,36,PHOTO,2023-11-13 00:00:00+00:00
3,0050fb0c-d51f-41c0-a779-cdf9b383ff80,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-11-13T00:00:00+00:00,Charts,10,Sans Coeur,2023-11-13 00:00:00+00:00
4,006fc41b-1998-42d0-85ff-193ce3e4d97f,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,33,Rolls Royce,2023-10-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...
1452,ff257c2a-450d-11ea-8546-801844ee83a6,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-07-26T00:00:00+00:00,Charts,45,Peut-être,2024-07-26 00:00:00+00:00
1453,ff318af0-3eb5-4996-a129-034fff30ca4f,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2024-06-10T00:00:00+00:00,Charts,48,GTA,2024-06-10 00:00:00+00:00
1454,ff3a1249-2ba5-4404-ae3c-13f10e6bd18f,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-12-11T00:00:00+00:00,Charts,26,Battitsu,2023-12-11 00:00:00+00:00
1455,ff3bdbfb-14c5-4f1f-b8e9-b87a818c1a39,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2024-09-23T00:00:00+00:00,Charts,26,CABREL,2024-09-23 00:00:00+00:00


In [23]:
# Define the output file
output_file = "charts_tracks_info_23-24.csv"

# Check if the file already exists and load existing data
if os.path.exists(output_file):
    api_results_df = pd.read_csv(output_file)
else:
    api_results_df = pd.DataFrame()

# Create a set of already processed track IDs to avoid duplicates
processed_tracks = set(api_results_df['track_uuid'].unique()) if not api_results_df.empty else set()

# List to store new results temporarily
new_results = []

for index, row in tqdm(first_appearance_df.iterrows(), total=len(first_appearance_df), desc="Processing rows"):
    playlist_id = row['playlist_id']
    playlist_name = row['playlist_name']
    collection_date = row['collection_date']
    playlist_type = row['playlist_type']
    track_position = row['track_position']
    track_name = row['track_name']
    track_id = row['track_uuid']
    
    try:
        tracks = sdch.get_track_metadata(track_id=track_id)
        track_dates = tracks.get("object", {}).get("releaseDate")
        track_features = tracks.get("object", {}).get("audio")

        artist_names = []
        artist_ids = []

        for artist in tracks.get("object", {}).get("artists", []):
            artist_names.append(artist.get("name"))
            artist_ids.append(artist.get("uuid"))

        label_names = []
        label_types = []

        for label in tracks.get("object", {}).get("labels", []):
            label_names.append(label.get("name"))
            label_types.append(label.get("type"))
        
        track_genres = []
        
        for genre in tracks.get("object", {}).get("genres", []):
            track_genres.append(genre.get("root"))

        track_data = ({
            "playlist_id": playlist_id,
            "playlist_name": playlist_name,
            "collection_date": collection_date,
            "playlist_type": playlist_type,
            "track_position": track_position,
            "track_name": track_name,
            "track_uuid": track_id,
            "artist_name": artist_names,
            "artist_id": artist_ids,
            "label_name": label_names,
            "label_type": label_types,
            "track_date": track_dates,
            "track_genres": track_genres,
            "track_features": track_features
        })

        # Append to the new results
        new_results.append(track_data)
        
        # Append directly to the file
        pd.DataFrame([track_data]).to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)

    except Exception as e:
        print(f"Error processing track {track_id}: {e}")
        continue

# Convert the new results to a DataFrame and combine with existing data if needed
if new_results:
    new_results_df = pd.DataFrame(new_results)
    api_results_df = pd.concat([api_results_df, new_results_df], ignore_index=True)

# Display the DataFrame
display(api_results_df)


Processing rows: 100%|██████████| 1457/1457 [05:12<00:00,  4.66it/s]


,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_uuid,artist_name,artist_id,label_name,label_type,track_date,track_genres,track_features
0,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-31T00:00:00+00:00,Charts,21,CARLO,000403f8-13a8-42af-8df6-f97b0f5372c7,[Josman],[11e81bc6-f131-49b8-8c1f-a0369fe50396],[Sideline],"[Other independent labels: ""Indie""]",2023-10-28T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.1, 'danceability': 0.56, 'e..."
1,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-05-02T00:00:00+00:00,Charts,40,L o s e M y M i n d,000e110b-3397-4082-982c-d85b0636629f,[PARTYNEXTDOOR],[11e81bcd-a0ae-86c0-8c3a-a0369fe50396],"[OVO, Santa Anna Label Group]","[Other independent labels: ""Indie"", unknown]",2024-04-11T00:00:00+00:00,"[r&b, funk & soul]","{'acousticness': 0.06, 'danceability': 0.43, '..."
2,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-11-13T00:00:00+00:00,Charts,36,PHOTO,002bed21-2779-4169-a6bc-9b57cbb437c4,"[Hös Copperfield, WeRenoi]","[46d7a152-8c3f-11e9-a162-549f35161576, b95ebba...","[Play two, DIRECT SANS ESCALE]","[Believe, unknown]",2023-11-10T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.47, 'danceability': 0.73, '..."
3,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-11-13T00:00:00+00:00,Charts,10,Sans Coeur,0050fb0c-d51f-41c0-a779-cdf9b383ff80,"[Niska, Shay]","[11e81bcb-6ed3-6f64-a533-a0369fe50396, ab14843...","[Because, Jolie Garce]","[Other independent labels: ""Indie"", unknown]",2023-11-10T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.17, 'danceability': 0.77, '..."
4,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,33,Rolls Royce,006fc41b-1998-42d0-85ff-193ce3e4d97f,[WeRenoi],[b95ebba3-b5e9-467a-afc2-1d148fc1d991],"[PGP, PLR Music]","[unknown, unknown]",2023-09-22T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.65, 'danceability': 0.78, '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1452,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-07-26T00:00:00+00:00,Charts,45,Peut-être,ff257c2a-450d-11ea-8546-801844ee83a6,"[Youssef Swatt's, El Gaouli]","[11e81bbf-fb75-3b98-b718-a0369fe50396, 11e81bc...",[Act Right],"[Other independent labels: ""Indie""]",2020-04-03T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.67, 'danceability': 0.61, '..."
1453,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2024-06-10T00:00:00+00:00,Charts,48,GTA,ff318af0-3eb5-4996-a129-034fff30ca4f,[Jul],[11e81bc2-225b-e886-a984-a0369fe50396],[D'Or et de Platine],[Self released],2024-06-07T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.01, 'danceability': 0.82, '..."
1454,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-12-11T00:00:00+00:00,Charts,26,Battitsu,ff3a1249-2ba5-4404-ae3c-13f10e6bd18f,[Jul],[11e81bc2-225b-e886-a984-a0369fe50396],[D'Or et de Platine],[Self released],2023-12-08T00:00:00+00:00,[hip-hop & rap],"{'acousticness': 0.07, 'danceability': 0.87, '..."
1455,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2024-09-23T00:00:00+00:00,Charts,26,CABREL,ff3bdbfb-14c5-4f1f-b8e9-b87a818c1a39,"[Tiakola, Saaro]","[34982a68-67ee-11e9-b1ad-549f35141000, 3239546...","[M3lo World, Label Atlantic]","[unknown, unknown]",2024-09-19T00:00:00+00:00,[],"{'acousticness': 0.44, 'danceability': 0.72, '..."


In [24]:
# Merge the two df
df = df.merge(
    api_results_df[[
        'track_uuid', 'artist_name', 'artist_id',
        'label_name', 'label_type','track_date', 
        'track_features', 'track_genres'
    ]],
    on='track_uuid',
    how='left'
)

df.to_csv('charts_tracks_info_23-24.csv', index=False)

# Display the updated DataFrame
display(df)


,playlist_id,playlist_name,collection_date,playlist_type,track_position,track_name,track_uuid,collection_date_clean,artist_name,artist_id,label_name,label_type,track_date,track_features,track_genres
0,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,1,Petit génie,5c0b0187-6e00-4499-b310-36e730252d13,2023-10-01 00:00:00+00:00,"[Abou Debeing, Alonzo, Imen Es, lossa, Jungeli]","[11e81bc3-36f8-dd20-b110-a0369fe50396, 11e83fe...","[2054 Records, Fulgu Prod, Next Génération]","[unknown, Other independent labels: ""Indie"", u...",2023-08-04T00:00:00+00:00,"{'acousticness': 0.13, 'danceability': 0.8, 'e...",[hip-hop & rap]
1,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,2,Casanova,be8682de-a811-4754-b310-9c816d3d6b56,2023-10-01 00:00:00+00:00,"[Soolking, Gazo]","[11e81bba-d00e-f2d2-a406-a0369fe50396, 59a6ff7...",[Pandor Music],[unknown],2023-07-06T00:00:00+00:00,"{'acousticness': 0.68, 'danceability': 0.8, 'e...","[hip-hop & rap, r&b, funk & soul]"
2,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,3,Saiyan,f1928fd5-134e-444e-a66c-7b5541a9f23f,2023-10-01 00:00:00+00:00,"[Heuss L'enfoiré, Gazo]","[11e81bce-22bb-888e-9f62-a0369fe50396, 59a6ff7...","[150 Prod, Straw Production]","[unknown, unknown]",2023-06-02T00:00:00+00:00,"{'acousticness': 0.36, 'danceability': 0.78, '...",[hip-hop & rap]
3,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,4,LAISSE MOI,065bda79-ba47-41cb-b44f-487741dc2ab9,2023-10-01 00:00:00+00:00,[KeBlack],[11e81bc8-7bb4-47b0-ac67-a0369fe50396],[Dayeden],[unknown],2023-09-08T00:00:00+00:00,"{'acousticness': 0.46, 'danceability': 0.77, '...",[hip-hop & rap]
4,11e84493-6b0e-6dc0-a8d5-a0369fe50396,Top 50 - France,2023-10-01T00:00:00+00:00,Charts,5,Meridian,a899c70e-bcfd-42e3-80b7-0d0280c63c43,2023-10-01 00:00:00+00:00,"[Dave, Tiakola]","[11e81bc7-3e47-39ce-afd6-a0369fe50396, 34982a6...","[Neighbourhood, Dave]","[Other independent labels: ""Indie"", Self relea...",2023-08-24T00:00:00+00:00,"{'acousticness': 0.23, 'danceability': 0.91, '...",[hip-hop & rap]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36590,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-09-30T00:00:00+00:00,Charts,46,L'avalanche,4cbd8b62-af47-4741-bf00-a9aae93c8f3a,2024-09-30 00:00:00+00:00,[Saïf],[e6706e43-e78d-43dc-b845-51505526bca7],[84 City],[unknown],2024-06-28T00:00:00+00:00,"{'acousticness': 0.47, 'danceability': 0.55, '...",[hip-hop & rap]
36591,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-09-30T00:00:00+00:00,Charts,47,Satanana (feat. DRE-A),4a630724-815e-41ae-8350-3f0f6f3c08a8,2024-09-30 00:00:00+00:00,"[DRE-A, EBOLOKO.]","[c59893b1-6ad6-49f7-b2bc-ba199f8a91f1, 17ae656...","[Sony, LABEL WAVE]","[Sony, unknown]",2024-08-23T00:00:00+00:00,"{'acousticness': 0.64, 'danceability': 0.63, '...",[african]
36592,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-09-30T00:00:00+00:00,Charts,48,Hm Hm hm,af61d486-4b1b-48a2-86f4-e0940116fa2d,2024-09-30 00:00:00+00:00,[Youka],[11e81bbd-e6ed-7430-8e61-a0369fe50396],[JAT KISS],[unknown],2024-07-05T00:00:00+00:00,"{'acousticness': 0.5, 'danceability': 0.57, 'e...","[r&b, funk & soul]"
36593,11e84493-6b26-e814-a42e-a0369fe50396,Viral 50 - France,2024-09-30T00:00:00+00:00,Charts,49,GOD BLESS - Ah la vie qu'on mène,e8e17944-43f1-4f42-9f2e-6e9cd94c9c03,2024-09-30 00:00:00+00:00,"[Krys, YSN]","[11e81bc1-c3c8-acbe-b9cf-a0369fe50396, 9446d8a...",[STEP OUT PRODUCTIONS],[unknown],2024-06-28T00:00:00+00:00,"{'acousticness': 0.14, 'danceability': 0.97, '...",[hip-hop & rap]
